In [1]:
import networkx as nx
import plotly.graph_objects as go
import numpy as np

In [2]:
# Create empty graph
G = nx.Graph()

# Add vertices in concentric circles
for r in range(1, 15):
    # Number of vertices doubles every 2 radii
    n_vertices = 4 * (2 ** ((r-1)//2))
    
    # Calculate angles for evenly spaced vertices
    angles = np.linspace(0, 2*np.pi, n_vertices, endpoint=False)
    if r % 2 == 1:
        angles += np.pi / n_vertices

    # Add vertices with positions and color based on radius parity
    for i in range(n_vertices):
        vertex = (r, i)  # (radius, index) tuple
        x = (r + 1) // 2 * np.cos(angles[i])
        y = (r + 1) // 2 * np.sin(angles[i])
        color = '#1f77b4' if r % 2 == 1 else '#ff7f0e'
        G.add_node(vertex, pos=(x,y), color=color)

    if r > 1:
        if r % 2 == 1: 
            for i in range(n_vertices):
                G.add_edge((r, i), (r-1, ((i + 1) % n_vertices) // 2), length=1)
        else:
            for i in range(n_vertices):
                G.add_edge((r, i), (r-1, (i % n_vertices)), length=1)
                G.add_edge((r, i), (r-1, ((i - 1) % n_vertices)), length=1)

# Get initial positions
pos_initial = nx.get_node_attributes(G, 'pos')

# Apply spring layout with initial positions
# pos = nx.spring_layout(G, pos=pos_initial, weight='length')
pos = pos_initial

In [3]:
def visualize_graph(G):
    # Get positions from graph
    pos = nx.get_node_attributes(G, 'pos')
    
    # Create edge coordinates
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    # Create node coordinates, colors and labels
    node_x = [pos[node][0] for node in G.nodes()]
    node_y = [pos[node][1] for node in G.nodes()]
    node_colors = [G.nodes[node]['color'] for node in G.nodes()]
    node_text = [f'({node[0]}, {node[1]})' for node in G.nodes()]

    # Create plot
    fig = go.Figure()

    # Add edges
    fig.add_trace(go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines'))

    # Add nodes
    fig.add_trace(go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        text=node_text,
        marker=dict(
            size=8,
            color=node_colors,
            line_width=2)))

    # Update layout
    fig.update_layout(
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20,l=5,r=5,t=40),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        width=800,
        height=800
    )
    
    return fig

In [4]:
fig = visualize_graph(G)
fig.write_html('./1d_mera.html')
fig.show()

In [5]:
def visualize_shortest_paths(G, src, dest):
    # Get base visualization
    fig = visualize_graph(G)
    
    # Get all shortest paths between src and dest
    shortest_paths = list(nx.all_shortest_paths(G, src, dest))
    
    # Get positions from graph
    pos = nx.get_node_attributes(G, 'pos')
    
    # Define a list of colors for different paths
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'cyan', 'magenta']
    
    # For each path, create and add a trace with a different color
    for i, path in enumerate(shortest_paths):
        # Create coordinates for highlighted edges
        highlight_edge_x = []
        highlight_edge_y = []
        
        # Get edges for this path
        path_edges = list(zip(path[:-1], path[1:]))
        
        # Add highlighted edges
        for edge in path_edges:
            x0, y0 = pos[edge[0]]
            x1, y1 = pos[edge[1]]
            highlight_edge_x.extend([x0, x1, None])
            highlight_edge_y.extend([y0, y1, None])

        # Add highlighted edges trace with unique color
        color = colors[i % len(colors)]  # Cycle through colors if more paths than colors
        fig.add_trace(go.Scatter(
            x=highlight_edge_x, y=highlight_edge_y,
            line=dict(width=2, color=color),
            hoverinfo='none',
            mode='lines'))
    
    # Get the length of shortest path (all paths have same length)
    path_length = len(shortest_paths[0]) - 1 if shortest_paths else 0
    
    return fig, path_length

src = (13, 0)
dest = (13, 81)
fig, length = visualize_shortest_paths(G, src, dest)
fig.show()
fig.write_html('../images/1d_mera_shortest_path.html')
print(length)

22


In [7]:
# Create empty graph
G = nx.Graph()

# Add vertices in concentric circles
for r in range(1, 8):
    n_vertices = 2 * 2 ** r 
    angles = np.linspace(0, 2*np.pi, n_vertices, endpoint=False)
    # angles += np.pi / n_vertices + np.pi / 4

    # Add vertices with positions and color based on radius parity
    for i in range(n_vertices):
        vertex = (r, i)  # (radius, index) tuple
        x = r * np.cos(angles[i])
        y = r * np.sin(angles[i])
        color = '#ff7f0e'
        G.add_node(vertex, pos=(x,y), color=color)

    for i in range(n_vertices):
        if r > 1:
            G.add_edge((r, i), (r-1, i // 2))
        G.add_edge((r, i), (r, (i+1) % n_vertices))
        G.add_edge((r, i), (r, (i-1) % n_vertices))

# Get initial positions
pos_initial = nx.get_node_attributes(G, 'pos')
fig = visualize_graph(G)
fig.write_html('../images/1d_mera_qisom.html')
fig.show()